# Poročilo
## Priporočilni sistem pri predmetu OS

Avtor: Jure Časar


Za nalogo je bilo potrebno implementirati priporočilni sistem na izbranih podatkih **MovieLens**.
Implementiranih je več načinov napovedovanja(naključno, s povprečjem, najbolj gledanih filmov, kontroverznih filmov, glede na podobnost ter z metodo SlopeOne)

Rešene naloge:
 - Branje ocen
 - Branje filmov
 - Naključni prediktor
 - Priporočanje
 - Napovedovanje s povprečjem
 - Priporočanje najbolj gledanih filmov
 - Priporočanje kontroverznih filmov
 - Napovedovanje ocen s podobnostjo med produkti
 - Najbolj podobni filmi
 - Priporočanje glede na trenutno gledano vsebino
 - Priporočilo zase
 - Napovedovanje z metodo SlopeOne

In [1]:
import numpy as np
import pandas as pd

### Branje ocen in filmov
Na začetku je potrebno podatke prebrati in shraniti v nek objekt. To sem naredil s kodo spodaj.

#### UserItemData
Pri branju ocen je možno določiti začetni in končni datum, med katerima želimo prebrati ocene. Možno je tudi določiti minimalno število ocen, ki jih film lahko ima, da je vsebovan. Razred **UserItemData**, s katerim beremo ocene, vsebuje tudi metodo **movies_from_user(self, uid)**, ki glede na id uporanika, vrne vse filme, ki jih je ocenil.

Za filtriranje glede na datum, sem prvo odstranil vrstice glede na leto, potem mesec in potem dan.
Na koncu sem vrnil le ocene tistih filmov, ki imajo zadostno število ocen.


In [2]:
class UserItemData:
    
    def __init__(self,path,start_date=None,end_date=None,min_ratings=0):
        self.data = pd.read_csv(path,sep="\s+")
        
        if start_date is not None: # Take into account start_date
            d , m, y = start_date.strip().split(".")
            self.data = self.data.loc[ (self.data["date_year"]   >= int(y)) ]
            self.data = self.data.loc[~((self.data["date_year"] == int(y)) & (self.data["date_month"] < int(m)))]
            self.data = self.data.loc[~((self.data["date_year"] == int(y)) & (self.data["date_month"] == int(m)) 
                                        & (self.data["date_day"] < int(d)))]
        
        if end_date is not None: # Take into account end_date
            d , m, y = end_date.strip().split(".")
            self.data = self.data.loc[ (self.data["date_year"] <= int(y)) ]
            self.data = self.data.loc[~((self.data["date_year"] == int(y)) & (self.data["date_month"] > int(m)))]
            self.data = self.data.loc[~((self.data["date_year"] == int(y)) & (self.data["date_month"] == int(m)) 
                                        & (self.data["date_day"] >= int(d)))]
        
        # Exclude movies with number of ratings < min_ratings
        nr = self.data["movieID"].value_counts()
        self.data = self.data.loc[self.data["movieID"].isin(nr.loc[(nr > min_ratings)].index.values)]
        
    def nratings(self):
        return len(self.data.index)
    
    def movies_from_user(self, uid): # Returns all movies from user
        user_mask = self.data["userID"] == uid
        return self.data["movieID"].loc[user_mask]

#### MovieData
Podatke o filmih beremo s pomočjo razreda **MovieData**. Tu preberemo le stolpca _id_ ter _title_.
Razred vsebuje tudi metodo **get_title(self, id)**, ki vrne naslov filma glede na podan id.

In [3]:
class MovieData:
    def __init__(self,path):
        self.data =  pd.read_csv(path,sep="\t+",usecols=["id","title"],engine="python")
    
    def get_title(self,id): # Tries to find the title of the given movie id
        try:
            return self.data["title"].loc[self.data["id"] == id].values[0]
        except IndexError:
            return None        

### Priporočanje
Za priporočanje je bil ustvarjen razred **Recommender**. Ta bo v nadaljevanju uporabljen za priporočanje filmov glede na različne načine napovedovanja.

Ko ustvarimo **Recommender**, mu zraven podamo tudi način napovedovanja (to je nek razred, ki vsebuje metodi _fit_ ter _predict_).

Metoda **fit** znotraj **Recommender** enostavno sprejme ocene filmov, jih shrani, ter pokliče _fit_ metodo od podanega razreda za napovedovanje.

Metoda **recommend** znotraj **Recommender** sprejme id uporabnika za katerega bo priporočala, število filmov, ki jih naj priporoči ter podatek ali naj priporoči že pogledane filme.
Znotraj metode se pokliče _predict_ od podanega razreda za napovedovanje, potem se pridobljene podatke uredi padajoče, ter po potrebi tudi odstrani filme, ki jih je uporabnik že gledal.
Na koncu se vrne prvih _n_ filmov.

In [4]:
class Recommender:
    def __init__(self, predictor):
        self.predictor = predictor
        
    def fit(self, X):
        self.uim = X
        self.predictor.fit(X)

    def recommend(self, userID, n=10, rec_seen=True):
        # Get predictions
        preds = self.predictor.predict(userID)
        # Sort predictions
        rec_movies = sorted(preds.items(), key=lambda x: x[1], reverse=True)
        if not rec_seen:
            # Exclude seen movies
            rec_movies = [m for m in rec_movies if m[0] not in self.uim.movies_from_user(userID).values]
        return dict(rec_movies[:n]) # Return a slice of first n predictions

## Razredi za napovedovanje (prediktorji)
V nadaljevanju bomo šli skozi različne načine napovedovanja. Vsak način napovedovanja bo prvo opisan, potem bo sledila koda same implementacije, potem pa primer priporočanja filmov za uporabnika z id=78.

Vsak **prediktor** vsebuji metodi **fit** in **predict**. Metoda **predict** vrne slovar<film, ocena>.

### Naključni prediktor
Naključni prediktor v samem srcu vsem filmom le določi naključno oceno. Naključno oceno lahko omejimo s parametri v konstruktorju.

Napovedovanje z uporabo naključnega prediktorja ni praktično, kaj šele personalizirano. Nam pa lepo pokaže ogrodje implementacije prediktorjev za naprej.

In [5]:
class RandomPredictor:
    def __init__(self, min_grade=0, max_grade=5):
        self.min = min_grade
        self.max = max_grade
        
    def fit(self,X):
        self.movies = X.data["movieID"].values
        
    def predict(self,uid):
        preds = dict()
        for m in self.movies:
            # Generate random grade for each movie
            preds[m] = np.random.randint(self.min, self.max+1)
        return preds

In [6]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat')

rec = Recommender(RandomPredictor(1,5))
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Film: The Crow, ocena: 5
Film: Ransom, ocena: 5
Film: Liar Liar, ocena: 5
Film: The Fifth Element, ocena: 5
Film: Back to the Future Part II, ocena: 5


### Prediktor s povprečjem
Pri prediktorju s povprečjem se za vsak film izračuna povprečje ocen, gle na naslednjo formulo: _avg = (vs + b * g-avg) / (n + b)_ ,

kjer je:
 - _vs_ vsota vseh ocen za ta film,
 - _n_ število ocen, ki jih je iflm dobil,
 - _g-avg_ povprečje čez vse filme,
 - _b_ parameter formule za povprečenje
 
Vsa čarovnija se dogaja znotraj metode **predict**. Prvotno pridobim povprečje čez vse filme. Potem seštejem ocene za vsak film, ter preštejem število ocen za vsak film. Te podatke združim v en DataFrame. Tako so na koncu število ocen in seštevek ocen za določen film v isti vrstici. Na koncu za vsak film (torej vsako vrstico) izračunam povprečje po formuli.

Napovedovanje s tem prediktorjem ni personalizirano. Ima pa proti prejšnemu, naključnemu prediktorju, vsaj nekaj pomena.

In [7]:
class AveragePredictor:
    def __init__(self, b):
        self.b = b
    
    def fit(self, X):
        self.data = X.data
        
    def predict(self, uid):
        # Get the overall average (global average)
        g_avg = self.data["rating"].mean()
        # Sum of ratings per movie
        m_sum = self.data[["movieID", "rating"]].groupby("movieID").sum().rename(columns={"rating":"sum"})
        # Number of ratings per movie
        m_count = self.data[["movieID", "rating"]].groupby("movieID").count().rename(columns={"rating":"count"})
        # Merge m_sum and m_count into dataframe
        data = m_sum.merge(right=m_count, how='inner', left_index=True, right_index=True)
        # Calculate an average for each movie(row in dataframe)
        self.preds = data.apply(lambda x: pd.Series([(x['sum'] + self.b * g_avg)/(x['count']+self.b)], 
                                                    index=['ocena']), axis=1)
        return dict(zip(self.preds.index, self.preds.ocena))

In [8]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat')

print("Priporočanje z parametrom b=0...")
rec = Recommender(AveragePredictor(0))
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))
    
print("\nPriporočanje z parametrom b=100...")
rec = Recommender(AveragePredictor(100))
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Priporočanje z parametrom b=0...
Film: Brother Minister: The Assassination of Malcolm X, ocena: 5.0
Film: Synthetic Pleasures, ocena: 5.0
Film: Gabbeh, ocena: 5.0
Film: Storefront Hitchcock, ocena: 5.0
Film: Ko to tamo peva, ocena: 5.0

Priporočanje z parametrom b=100...
Film: The Usual Suspects, ocena: 4.225944245560473
Film: The Godfather: Part II, ocena: 4.146907937910189
Film: Cidade de Deus, ocena: 4.116538340205236
Film: The Dark Knight, ocena: 4.10413904093503
Film: 12 Angry Men, ocena: 4.103639627096175


### Priporočanje najbolj gledanih filmov
Tu metoda **predict** le vrne število ocen za vsak film. Glede na to se potem tudi priporočajo filmi.



In [9]:
class ViewsPredictor:
    def fit(self,X):
        self.data = X.data
        
    def predict(self,uid):
        # Just return number of ratings for each movie
        return self.data["movieID"].value_counts()

In [10]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat')

rec = Recommender(ViewsPredictor())
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 1576
Film: The Lord of the Rings: The Two Towers, ocena: 1528
Film: The Lord of the Rings: The Return of the King, ocena: 1457
Film: The Silence of the Lambs, ocena: 1431
Film: Shrek, ocena: 1404


### Priporočanje kontroverznih filmov
Prediktor prejme parameter _n_, ki določa minimalno število ocen, ki jih mora film imeti, da je lahko sploh "kontroverzen".

Za mero smo vzeli standardno deviacijo.

Znotraj metode **predict** se prvotno odstrani filme, ki imajo manj ocen kot je bil podan parameter _n_.
Potem pa se ocene grupira po filmih ter izračuna standardno deviacijo.

Ta način priporočanjo ni personaliziran, pa vendarle je lahko uporaben za neko splošno priporočanje. Mene na primer bi zanimalo, kateri so najbolj kontroverzni filmi.

In [11]:
class STDPredictor:
    def __init__(self, n):
        self.n = n
    
    def fit(self,X):
        self.data = X.data
        
    def predict(self,uid):
        # Create a mask for movies with number of ratings > self.n
        nr = self.data["movieID"].value_counts()
        movies = nr.loc[(nr > self.n)].index
        mask = self.data["movieID"].isin(movies)
        # Group by movieID and calculate standard deviation
        self.preds = self.data[["movieID","rating"]].loc[mask].groupby("movieID").std()["rating"]
        return self.preds

In [12]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat')

rec = Recommender(STDPredictor(100))
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Film: Plan 9 from Outer Space, ocena: 1.3449520951495717
Film: The Passion of the Christ, ocena: 1.281493459525735
Film: The Texas Chainsaw Massacre, ocena: 1.235349321908819
Film: Jackass Number Two, ocena: 1.2189769976366684
Film: White Chicks, ocena: 1.1899581424297319


### Napovedovanje ocen s podobnostjo med produkti

Napovedovanje s podobnostjo med produkti je implementirano tako, da se v metodi **fit** zgradi matriko podobnosti. Potem pa v metodi **predict** izračuna napovedano oceno za vsak film glede na podobnost z filmi, ki jih je gledal.

Sama podobnost se izračuna po naslednji formuli: ![Cosine Similarity](./images/similarity_formula.png)

Ocena filma pa se izračuna tako: ![Prediction Score Formula](./images/predicted_score.png)

**Grajenje matrike podobnosti**. Na začetku podatke obrnemo, tako da stolpci predstavljajo ocene za določen film, vrstice pa ocene od določenega uporabnika. Tem ocenam odštejemo povprečje od uporabnika, ki je podal posamezno oceno. Potem pa za grajenje same matrike podobnosti iteriramo skozi vsak možen par filmov, ter za njiju izračunamo podobnost glede na zgornjo formulo.

**Napovedovanje ocen za vsak film**.
Ocena se izračuna po zgornji formuli. Sestavljena je iz seštevka podobnosti med filmom, ki mu napovedujemo oceno ter vsemi filmi, ki jih je uporabnik gledal. Vsaka podobnost je še pomnožena z normalizirano oceno od uporabnika. Na koncu se ta seštevek deli z samim seštevkom vseh podobnosti in prišteje povprečje ocen uporabnika.

Prediktor vsebuje tudi metodo **get_most_similar_items(self, n=20)**, ki vrne _n_ najbolj podobnih filmov.
V primeru, da matrika podobnosti še ni bila izračunana, vrne prazen seznam. Če ne pa se sprehodi skozi zgornji trikotnik matrike(brez glavne diagonale), podobnosti si zapiše v seznam, ga sortira in vrne prvih _n_ parov filmov.

Prediktor lahko priporoča še _n_ najbolj podobnih filmov danemu filmu z metodo **similar_items(self, item, n)**.


In [13]:
class ItemBasedPredictor:
    def __init__(self, min_ratings=0, threshold=0):
        self.mr = min_ratings
        self.thr = threshold
        
    def get_most_similar_movies(self, n=20):
        if not hasattr(self, 'sm'):
            # Similarity matrix was not calculated yet @return empty list
            return []
        else:
            sims = []
            # Iterate through upper triangle (excludeing main diagonal) and collect all similarities
            for i, row in enumerate(self.sm.index[:-1]):
                for col in self.sm.columns[(i+1):]:
                    sims.append((col, row, self.sm[col][row]))
            # Sort similarities and slice only first n
            return sorted(sims, key=lambda x: x[2], reverse=True)[:n]
        
    def similar_items(self, item, n):
        if not hasattr(self, 'sm'):
            # Similarity matrix was not calculated yet @return empty list
            return []
        else:
            try:
                # Sort similarities of given movie @return first n+1 (excluding the very first (similarity(item,item)))
                return self.sm[item].sort_values(ascending=False)[1:(n+1)]
            except KeyError:
                # Movie not found in similarity matrix @return empty list
                return []
            
    def similarity(self, m1, m2):
        # @Return similarity between given movies
        return self.sm[m1][m2]
        
    def build_similarity_matrix(self, m):
        sm = pd.DataFrame(pd.DataFrame(), columns=m.columns, index=m.columns)
        
        for c1 in m.columns:
            r1 = m[c1].dropna().rename("r1")
            for c2 in m.columns:
                # inner join
                r2 = m[c2].dropna().rename("r2")
                r12 = pd.merge(left=r1, right=r2, how='inner', left_index=True, right_index=True)
                
                if len(r12.index) < self.mr: # if not enough ratings then similarity is 0
                    sm.at[c1, c2] = 0
                    continue
                
                if c1 == c2: # if same columns then similarity is 1
                    sm.at[c1, c2] = 1
                    continue
                
                # similarity calculation
                dot_product = r12["r1"] @ r12["r2"]
                norm_1 = np.linalg.norm(r12["r1"])
                norm_2 = np.linalg.norm(r12["r2"])
                similarity = dot_product/(norm_1 * norm_2) if (norm_1 * norm_2) != 0 else 0
                
                # if below threshold then similarity is 0
                sm.at[c1, c2] = similarity if similarity >= self.thr else 0
        self.sm = sm
    
    def fit(self, X):
        # Pivot table and normalize by subtracting ratings by their users average rating
        mr = X.data.pivot_table(index="userID", columns="movieID", values="rating")
        mr['avg'] = mr.mean(axis=1)
        self.urm = mr
        norm = mr.sub(mr['avg'], axis=0)
        del norm['avg']
        self.urm_norm = norm
        self.build_similarity_matrix(norm)   
    
    def predict(self, userID):
        ra = self.urm['avg'][userID]
        
        user_rated_movies = self.urm.loc[userID, :][:-1].dropna().index # movies rated by userID
        
        self.preds = dict()
        # Calculate prediction score for each movie
        for m in self.urm.columns[:-1]: # score(userID, m)
            s1 = 0
            s2 = 0
            for um in user_rated_movies:
                s = self.similarity(m, um)
                rating_scale = self.urm[um][userID] - ra
                s1 += s*rating_scale
                s2 += s
            score = (s1/s2)+ra
            self.preds[m] = score
        return self.preds

In [14]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat', min_ratings=1000)

pr = ItemBasedPredictor()
rec = Recommender(pr)
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Film: Shichinin no samurai, ocena: 4.3557347903101595
Film: The Usual Suspects, ocena: 4.3546817280678365
Film: The Silence of the Lambs, ocena: 4.335305303472516
Film: Sin City, ocena: 4.2786871668991004
Film: Monsters, Inc., ocena: 4.2175811369435205


**Izpišimo še 5 najbolj podobnih filmov**

Očitno obstajata 2 filma _Kill Bill: Vol. 2_ ali pa je to le napaka pri podatkih.

In [15]:
for m1, m2, sim in pr.get_most_similar_movies(5):
    print("Film1: {}\nFilm2: {}\npodobnost: {}\n\n".format(md.get_title(m1), md.get_title(m2), sim))

Film1: The Lord of the Rings: The Return of the King
Film2: The Lord of the Rings: The Two Towers
podobnost: 0.8439842148481416


Film1: The Lord of the Rings: The Two Towers
Film2: The Lord of the Rings: The Fellowship of the Ring
podobnost: 0.823188540176189


Film1: The Lord of the Rings: The Return of the King
Film2: The Lord of the Rings: The Fellowship of the Ring
podobnost: 0.8079374897442493


Film1: Kill Bill: Vol. 2
Film2: Kill Bill: Vol. 2
podobnost: 0.7372340224381029


Film1: Star Wars: Episode V - The Empire Strikes Back
Film2: Star Wars
podobnost: 0.702132113222032




**Priporočajmo še filme glede na ravno pogledan film: The Lord of the Rings: The Fellowship of the Ring**

In [16]:
for idmovie, val in pr.similar_items(4993, 5).items():
    print("Film: {}, ocena: {}".format(md.get_title(idmovie), val))

Film: The Lord of the Rings: The Two Towers, ocena: 0.823188540176189
Film: The Lord of the Rings: The Return of the King, ocena: 0.8079374897442493
Film: Star Wars: Episode V - The Empire Strikes Back, ocena: 0.23961943073496464
Film: Star Wars, ocena: 0.2196558652707407
Film: The Matrix, ocena: 0.2151555270688024


### Napovedovanje z metodo SlopeOne

Metoda SlopeOne je poenostavitev linearne regresije in se tudi v večini primerov izkaže za boljšo metodo.

V moji implementaciji sem prvo tabelo obrnil tako, da so stolpci ocene od določenega filma, vrstice pa ocene od določenega uporabnika.
Potem ko sem za posamezen film računal oceno, sem za vsak par filmov(med filmom, ki mu računamo oceno in filmom, ki ga je uporabnik ocenil) naredil naslednje:
 - pridobil sem ocene od obeh filmov, in sicer brez NaN vrednosti
 - ta dva seznama sem združil glede na uporabnike, ki so ocenili. Tako so ostali uporabniki (vrstice), ki so ocenili oba filma
 - potem sem izračunal povprečje, ki sem ga množil s številom uporabnikov, ki so ocenili oba filma
 - temu sem prištel še oceno od uporabnika, za katerega napovedujemo
 - si zapomnil število uporabnikov, ki so ocenili oba filma

Tako sem za vsak film, ki se mu računa oceno, seštel izračune od vseh parov filmov ter jih delil z skupnim številom uporabnikov, ki so takrat ocenili tisti par filmov.


In [17]:
class SlopeOnePredictor:
    def fit(self, X):
        # Pivot table
        self.mr = X.data.pivot_table(index="userID", columns="movieID", values="rating")
    
    def predict(self, userID):
        # Get user rated and unrated movies
        user_rated_movies = self.mr.loc[userID, :].dropna()
        user_unrated_movies = self.mr.loc[userID, ~self.mr.loc[userID, :].notna()]
        
        preds = []
        for unrated_movie in user_unrated_movies.index: # Predicting scores for unrated movies ...
            score = 0
            n = 0
            mr1 = self.mr[unrated_movie].dropna().rename("r1")
            for rated_movie in user_rated_movies.index: # ...with help of rated movies
                sub_score = self.mr.loc[userID, rated_movie]
                mr2 = self.mr[rated_movie].dropna().rename("r2")
                
                # Inner join on both movies -> only ratings from users who rated both movies
                mr12 = pd.merge(left=mr1, right=mr2, how='inner', left_index=True, right_index=True)
                diff = mr12["r1"] - mr12["r2"]
                sub_score += diff.mean()
                sub_score *= len(mr12.index)
                n += len(mr12.index)
                score += sub_score
            score  = score/n if n != 0 else 0
            preds.append(score)
        to_append = pd.Series(preds, index=user_unrated_movies.index)
        self.preds = user_rated_movies.append(to_append)
        return dict(self.preds)

In [18]:
md = MovieData('movielens/movies.dat')
uim = UserItemData('movielens/user_ratedmovies.dat', min_ratings=1000)

rec = Recommender(SlopeOnePredictor())
rec.fit(uim)
for movie_id, val in rec.recommend(78, n=5, rec_seen=False).items():
    print("Film: {}, ocena: {}".format(md.get_title(movie_id), val))

Film: The Usual Suspects, ocena: 4.325079182263173
Film: The Lord of the Rings: The Fellowship of the Ring, ocena: 4.155293229840448
Film: The Lord of the Rings: The Return of the King, ocena: 4.153135076202185
Film: The Silence of the Lambs, ocena: 4.127978169643881
Film: Shichinin no samurai, ocena: 4.119790444913598


### Priporočilo zase

Da lahko sebi priporočim nekaj filmov, sem spodaj vstavil še svojih 19 ocen.

In [19]:
uim_my = UserItemData('movielens/user_ratedmovies.dat', min_ratings=1000) 
my_ratings = [(20,4),(480,4.5),(1270,3.5),(6539,5),(1196,3.5),(260,3),(541,4),(2571,5),(8961,4),(1240,4),(589,5),(1036,5),
             (1721,5),(648,3.5),(5349,3.5),(2628,3),(597,2),(1291,3.5),(457,4.5)]
for movie, rating in my_ratings:
    uim_my.data = uim_my.data.append([{"userID":1, "movieID": movie, "rating":rating,
                                "date_day": 1, "date_month": 1, "date_year": 2021, "date_hour": 1,
                                "date_minute": 1, "date_second": 1}], ignore_index=True)

In [20]:
rec = Recommender(ItemBasedPredictor())
rec.fit(uim_my)
for idmovie, val in rec.recommend(1, n=10, rec_seen=False).items():
    print("Film {}: {}, ocena: {}".format(idmovie, md.get_title(idmovie), val))

Film 1704: Good Will Hunting, ocena: 5.0
Film 7438: Kill Bill: Vol. 2, ocena: 5.0
Film 6874: Kill Bill: Vol. 2, ocena: 4.810554185599607
Film 2762: The Sixth Sense, ocena: 4.7157258968696745
Film 47: Shichinin no samurai, ocena: 4.419897285820535
Film 2028: Saving Private Ryan, ocena: 4.379178024873356
Film 2858: American Beauty, ocena: 4.320662461778854
Film 593: The Silence of the Lambs, ocena: 4.291815661708156
Film 2959: Fight Club, ocena: 4.286300980895842
Film 32587: Sin City, ocena: 4.285364107974284
